Make sure to set the runtime to GPU: Runtime -> Change runtime type -> T4 GPU

You can upload your own images, then change the relevant code cells to load it and send it through the model.

In [1]:
# install the intrinsic decomposition repo from github
!pip install https://github.com/compphoto/Intrinsic/archive/main.zip

     - 38.2 MB 28.7 MB/s 0:00:01
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/MiDaS (to revision master) to /tmp/pip-install-3c_eisxa/altered-midas_cef02ca7dd864a539852f5fa6983a8c2
  Running command git clone --filter=blob:none --quiet https://github.com/CCareaga/MiDaS /tmp/pip-install-3c_eisxa/altered-midas_cef02ca7dd864a539852f5fa6983a8c2
  Resolved https://github.com/CCareaga/MiDaS to commit ca29fc7041dd11a2347d93046e758bb3c7adaac6
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/chrislib (to revision main) to /tmp/pip-install-3c_eisxa/chrislib_9030b857731648529f0d46983fe67cda
  Running command git clone --filter=blob:none --quiet https://github.com/CCareaga/chrislib /tmp/pip-install-3c_eisxa/chrislib_9030b857731648529f0d46983fe67cda
  Resolved https://github.com/CCareaga/chrislib to commit cf818e04e16495e0228d0a418733280538a5857b
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4

In [3]:
import torch

# import some helper functions from chrislib (will be installed by the intrinsic repo)
from chrislib.general import show, view, invert
from chrislib.data_util import load_from_url

# import model loading and running the pipeline
from intrinsic.pipeline import load_models, run_pipeline
from chrislib.general import show, view, uninvert
from chrislib.data_util import load_image

# import model loading and running the pipeline
from intrinsic.pipeline import run_pipeline

import cv2
import math
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from skimage.transform import resize
import os
from chrislib.data_util import np_to_pil

In [4]:
# download the pretrained weights and return the model (may take a bit to download weights the first time)
intrinsic_model = load_models('v2')

Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v2.0/stage_0.pt" to /root/.cache/torch/hub/checkpoints/stage_0.pt
100%|██████████| 403M/403M [00:01<00:00, 256MB/s]
Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v2.0/stage_1.pt" to /root/.cache/torch/hub/checkpoints/stage_1.pt
100%|██████████| 81.8M/81.8M [00:00<00:00, 273MB/s]
Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v2.0/stage_2.pt" to /root/.cache/torch/hub/checkpoints/stage_2.pt
100%|██████████| 403M/403M [00:09<00:00, 45.0MB/s]
Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v2.0/stage_3.pt" to /root/.cache/torch/hub/checkpoints/stage_3.pt
100%|██████████| 403M/403M [00:06<00:00, 66.6MB/s]
Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v2.0/stage_4.pt" to /root/.cache/torch/hub/checkpoints/stage_4.pt
100%|██████████| 403M/403M [00:01<00:00, 261MB/s]
Downloading: "https://github.com/facebookresearch/WSL-Images/

In [8]:
# load the image to run through the pipeline
img = load_from_url('https://raw.githubusercontent.com/compphoto/Intrinsic/refs/heads/main/figures/canal.png')
input_folder = '/content/imgs/'  # 替换为你的输入文件夹路径
output_folder = '/content/out_imgs/'  # 替换为你的输出文件夹路径
for filename in os.listdir(input_folder):
    # 构建完整的文件路径
    file_path = os.path.join(input_folder, filename)

    # 检查文件是否是图像文件（如 .jpg、.png 等）
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        # 加载图像
        img = load_image(file_path)
        if img.shape[0]<=img.shape[1]:

            img = resize(img, (384, 512))
        else:
            img = resize(img, (512, 384))
        result = run_pipeline(
            intrinsic_model,
            img,
            device='cuda'
        )

        # 处理图像
        shd = 1 - invert(result['dif_shd'])
        alb = view(result['hr_alb'])

        # 自定义命名方案，例如：new_filename_1.png, new_filename_2.png, ...
        new_s = f'S_{filename}'
        output_paths = os.path.join(output_folder, new_s)
        new_r = f'R_{filename}'
        output_pathr = os.path.join(output_folder, new_r)

        shading=(view(shd)*255).astype(np.uint8)
        shading=cv2.cvtColor(shading, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_paths, shading)

        alb=(view(alb)*255).astype(np.uint8)
        alb=cv2.cvtColor(alb, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_pathr, alb)



        # 保存图像（使用 cv2 保存）
        # 如果要保存灰度图像，请确保数据格式为单通道
        print(f'Saved: {output_pathr}')

print('All images processed and saved successfully.')

Saved: /content/out_imgs/R_eopch039_75145..png
Saved: /content/out_imgs/R_eopch039_102957..png
All images processed and saved successfully.


In [ ]:
# run the image through the pipeline (runs all stages)
result = run_pipeline(
    intrinsic_model,
    img,
    device='cuda'
)

In [ ]:
img = result['image']
alb = view(result['hr_alb']) # gamma correct the estimated albedo
dif = 1 - invert(result['dif_shd']) # tonemap the diffuse shading
res = result['residual']

In [ ]:
show([img, alb, dif, res], size=(30, 7))